In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
data_path = "../gmm_dataset_c5k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
NUM_SEQS, N, D = Data.shape
K = 3 ## number of clusters
## Model Parameters
MCMC_SIZE = 2
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32
## Training Parameters
BATCH_SIZE = 20
NUM_BATCHES = int((NUM_SEQS / BATCH_SIZE))
NUM_EPOCHS = 500
LEARNING_RATE = 5 * 1e-4
CUDA = torch.cuda.is_available()
PATH = 'ag-os-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:0')
## wrap up the parameters
Train_Params = (NUM_EPOCHS, NUM_BATCHES, NUM_SEQS, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE)

In [3]:
from local_gibbs import *
from global_oneshot import *
from global_enc import *
## if reparameterize continuous variables
Reparameterized = True
# initialization
enc_z = Gibbs_z(K, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

models = (oneshot_eta, enc_eta, enc_z)

In [4]:
from eubo_ag import *
train(Eubo_cfz_os_eta, optimizer, Data, Train_Params, Model_Params, models)

epoch: 0\500 (13s),  symKL: 3720.272,  eubo: -11460.959,  elbo: -15181.230,  ess: 1.139,  kl_eta_ex: 1062.783,  kl_eta_in: 215.299,  kl_z_ex: 145.061,  kl_z_in: 120.632
epoch: 1\500 (12s),  symKL: 2619.924,  eubo: -10540.473,  elbo: -13160.397,  ess: 1.151,  kl_eta_ex: 621.059,  kl_eta_in: 209.402,  kl_z_ex: 103.698,  kl_z_in: 87.814
epoch: 2\500 (12s),  symKL: 2069.721,  eubo: -9945.848,  elbo: -12015.569,  ess: 1.174,  kl_eta_ex: 415.960,  kl_eta_in: 197.199,  kl_z_ex: 75.417,  kl_z_in: 62.799
epoch: 3\500 (12s),  symKL: 1693.024,  eubo: -9600.316,  elbo: -11293.340,  ess: 1.208,  kl_eta_ex: 296.052,  kl_eta_in: 195.147,  kl_z_ex: 53.838,  kl_z_in: 44.537
epoch: 4\500 (13s),  symKL: 1447.691,  eubo: -9345.037,  elbo: -10792.728,  ess: 1.237,  kl_eta_ex: 224.663,  kl_eta_in: 199.277,  kl_z_ex: 39.546,  kl_z_in: 33.572
epoch: 5\500 (13s),  symKL: 1258.554,  eubo: -9195.044,  elbo: -10453.598,  ess: 1.282,  kl_eta_ex: 181.223,  kl_eta_in: 209.686,  kl_z_ex: 31.163,  kl_z_in: 26.603
epoc

KeyboardInterrupt: 

In [ ]:
torch.save(enc_eta.state_dict(), '../weights/enc-eta-%s' + PATH)
torch.save(oneshot_eta.state_dict(), '../weights/oneshot-eta-%s' + PATH)

In [ ]:
BATCH_SIZE_TEST = 50
Model_Params_Test = (N, K, D, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE_TEST)
obs, metric_step, reused = test(Eubo_cfz_os_eta, Data, Train_Params, Model_Params_Test, models)
(q_eta, _, q_z, _, _, _) = reused

In [ ]:
%time plot_samples(obs, q_eta, q_z, PATH)

In [ ]:
incremental_gap = symkl_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')

In [ ]:
incremental_gap